# Clickstream Analysis using Apache Spark and Apache Kafka(or Message hub).

[Message Hub: Apache Kafka as a Service](https://developer.ibm.com/messaging/2016/03/14/message-hub-apache-kafka-as-a-service/), is well integrated into the IBM Data Science Experience.


Before running the notebook, you will need to setup a [Message hub](https://developer.ibm.com/messaging/message-hub/) service. 

**Note:** Message hub is a paid service.

* For creating a Message hub service, go to `Data services-> Services` tab on the dashboard. Select the option to create a message hub service following the on screen instructions. Post creating service instance, select it and create a topic `clicks` with all defaults or as per your preference.

* Once the service is running, it has to be added to the current notebook. For this, first we need to create a connection for this message hub service instance. Go to `Data services-> connections` tab on dashboard and create new connection filling in the details and referring to the above created service instance in the Service instance section and then select topic `clicks`. Once done, go to `Assets` tab on the project dashboard and then click `+New data asset`. Then locate the created message hub service connection under the connections tab and click `Apply`.

* Once the service is added to the notebook, credentials to access it can be auto inserted. Please follow comments in the next step, for instructions on how to insert credentials. Once the credentials are inserted it is ready for execution.


### Credentials Section. 

In [1]:
 // In a Project create a Connection to a Message Hub topic and then in a Notebook in the Project use 'Insert to Code'
// to get the connection info pasted into the code cell. And rename it to credentials_1.

 // Just like this: 
// @hidden_cell
/*var credentials_1 = scala.collection.mutable.HashMap[String, String](
  "instance_id"->"***",
  "mqlight_lookup_url"->"***",
  "api_key"->"***",
  "kafka_admin_url"->"***",
  "kafka_rest_url"->"***",
  "kafka_brokers_sasl"->"ip:port",
  "user"->"***",
  "password"->"""****""",
  "topic"->"test"
)*/
// @hidden_cell


spark.version

Waiting for a Spark session to start...

2.1.0

Specify the schema of the incoming wikipedia clickstream and the parse method:

In [2]:
import scala.util.Try


In [3]:
case class Click(prev: String, curr: String, link: String, n: Long)


In [4]:
def parseVal(x: Array[Byte]): Option[Click] = {
    val split: Array[String] = new Predef.String(x).split("\\t")
    if (split.length == 4) {
      Try(Click(split(0), split(1), split(2), split(3).toLong)).toOption
    } else
      None
  }

In [5]:
val user = credentials_1("user")
val pass = credentials_1("password")
val topic = credentials_1("topic")
val saslConfig =
    s"""org.apache.kafka.common.security.plain.PlainLoginModule required
      |debug=true
      |username="$user"
      |password="$pass";""".stripMargin


Setup structured streaming to read from Kafka:

In [6]:
val records = spark.readStream.format("kafka").option("kafka.sasl.jaas.config", saslConfig).option("subscribe", topic).option("kafka.sasl.mechanism", "PLAIN").option("kafka.security.protocol", "SASL_SSL").option("failOnDataLoss", "false").option("kafka.bootstrap.servers", credentials_1("kafka_brokers_sasl")).load()


Process the records:

In [7]:
val messages = records.select("value").as[Array[Byte]].flatMap(x => parseVal(x)).groupBy("curr").agg(Map("n" -> "sum")).sort($"sum(n)".desc)

Output to the console and start streaming:

In [15]:
val query = messages.writeStream.outputMode("complete").option("truncate", "false").format("console").start()


-------------------------------------------
Batch: 0
-------------------------------------------
+----+------+
|curr|sum(n)|
+----+------+
+----+------+

-------------------------------------------
Batch: 7
-------------------------------------------
+---------+------+
|curr     |sum(n)|
+---------+------+
|Aristotle|108441|
+---------+------+

-------------------------------------------
Batch: 8
-------------------------------------------
+--------------------------------------------+------+
|curr                                        |sum(n)|
+--------------------------------------------+------+
|Aristotle                                   |108441|
|Monday_Night_Massacre                       |49366 |
|Fin_(Syd_album)                             |762   |
|#################                           |313   |
|Binghamton_Devils                           |280   |
|Fueler                                      |254   |
|Alpine_skiing_at_the_2017_Winter_Universiade|193   |
|Mudyug_concentr